In [38]:
#### finger_detected_circle_moving_v1_02162019.mp4 script ####
#### This one detects a circle resulting from the finger curl
import cv2
import numpy as np
import math

center = [] #place to store all the data
frame_index = 0

#set up video capture
vid = cv2.VideoCapture('openloop_pwm_mask.mp4')
out = None
while(True):
    #print(vid.isOpened())
    #read in video
    ret, frame = vid.read() 
    if ret:
        frame_dimension = frame.shape
        frame_height = frame_dimension[0]
        frame_width = frame_dimension[1]
        
        #setup the video file format of the output
        if out is None:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use the lower case
            out = cv2.VideoWriter('finger_detected_circle_moving_v1_02132021.mp4', 
                              fourcc, 20.0, (frame_width, frame_height), True)
    
        #convert video from BGR to Grayscale
        gray_vid = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #Apply median blur to the grayscaled video - parameters: input video, kernel size
        gray_vid = cv2.medianBlur(gray_vid, 21)
        
        #Apply Hough Circle Transform
        circles = cv2.HoughCircles(gray_vid,cv2.HOUGH_GRADIENT,1,20,
                                   param1=110,param2=30,minRadius=0,maxRadius=0)
        #Set circles variable equal to itself cast to a numpy array of uint16 data type, with rounding to 0 decimals
        circles = np.uint16(np.around(circles))
        
        #Convert grayscaled video to BGR for drawing the discovered circles and their center point
        bgr_vid = cv2.cvtColor(gray_vid, cv2.COLOR_GRAY2BGR)
        
        for i in circles[0,:]: #for i in circles[0,:]:
            center.append([frame_index, i[0], i[1]])
           # draw the outer circle
            cv2.circle(bgr_vid,(i[0],i[1]),i[2],(0,255,0),2)
           # draw the center of the circle
            cv2.circle(bgr_vid,(i[0],i[1]),2,(0,0,255),3)
            
        frame_index = frame_index + 1    
        
        out.write(bgr_vid) #generate output videos
    else:
        break
np.savetxt("video1.csv", center, delimiter=",", fmt='%d', comments = '')
out.release()
vid.release()
cv2.destroyAllWindows()

In [50]:
import csv
import numpy as np

# csv file name
filename = "video1.csv"

#intializting the titles and rows list
fields = []
rows = []
res = []

#the correct tracking points' coordinates from last frame, they'll be updated every frame
prev_x0 = 0
prev_y0 = 0
prev_x1 = 0
prev_y1 = 0


#reading csv file
with open(filename, 'r') as csvfile:
    #creating csv reader object
    csvreader = csv.reader(csvfile)
    
    #rows = next(csvreader)
    
    for row in csvreader:  #take the csv file into a array
        rows.append(row)

for row in rows[0:]:
    if(prev_x0 == 0):
        prev_x0 = int(row[1])
        prev_y0 = int(row[2])
        res.append([int(row[0]), int(row[1]), int(row[2])])
    elif prev_x1 == 0:
        prev_x1 = int(row[1])
        prev_y1 = int(row[2])
        res.append([int(row[0]), int(row[1]), int(row[2])])
    elif(abs(int(row[1])-prev_x0)<=150 and abs(int(row[2])-prev_y0)<=150):
        prev_x0 = int(row[1])
        prev_y0 = int(row[2])
        res.append([int(row[0]), int(row[1]), int(row[2])])
    elif(abs(int(row[1])-prev_x1)<=150 and abs(int(row[2])-prev_y1)<=150):
        prev_x1 = int(row[1])
        prev_y1 = int(row[2])
        res.append([int(row[0]), int(row[1]), int(row[2])])
        
        
np.savetxt("video_edit.csv", res, delimiter=",", fmt='%d', comments = '')

        
    